# TP 08 : Régularisation et sélection d'attributs


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

## I- Régularisation 

Ici, on va appliquer la régularisation sur la régression logistique.

### I-1- Fonctions utiles

In [ ]:
# une fonction qui normalise une matrice sur chaque colonne 
# dans l'entraînement, on calcule les moyennes et les déviations de chaque colonne
# dans le teste, on ne doit pas recalculer ces deux paramètres. 
# Mais, on doit utiliser ceux calculés dans l'entrainnement
def normaliser(X, mean=None, std=None): 
    if (mean is None) or (std is None): 
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0)
    X_norm = (X - mean)/std
    return X_norm, mean, std

# Préparer la matrice des caractéristiques pour l'entrainnement ou la prédiction
# On applique la normalisation sur chaque colonne (sauf si : norm=False)
# Ensuite on ajoute une colonne des 1 (sauf si : const=False)
def preparer(X, norm=True, const=True, mean=None, std=None): 
    X_pre = X.copy()
    if norm: 
        X_pre, mean, std = normaliser(X_pre, mean=mean, std=std)
    if const:
        X_pre = np.append(np.ones((X_pre.shape[0],1)), X_pre ,axis=1)
    return X_pre, mean, std

# Défininir des fonctions qui génèrent des vecteurs ou des matrices
# Ces fonctions seront utilisées pour générer les Thétas

def generer_zeros_1(nbr):
    return np.zeros(nbr)

def generer_uns_1(nbr):
    return np.ones(nbr)

def generer_aleatoire_1(nbr):
    return np.random.rand(nbr)

def afficher_2(X, Y, L=None, Y_labels=["Admis", "Non admis"], X_labels=["Note 1", "Note 2"]):
    oui = Y == 1
    plt.scatter(X[oui, 0], X[oui, 1], color="green", marker="o", label=Y_labels[0])
    plt.scatter(X[~oui, 0], X[~oui, 1], color="red", marker="x", label=Y_labels[1])
    if L is not None:
        x_values, y_values = L
        plt.plot(x_values, y_values, label="ligne de decision")
    plt.xlabel(X_labels[0])
    plt.ylabel(X_labels[1])
    plt.legend()
    plt.show()
    
# th0 + th1 * x1 + th2 * x2 = 0
def ligne_decision(X, Theta, mean=[0, 0], std=[1, 1]):
    X1 = [np.min(X[:,0]), np.max(X[:,0])]
    X2_min = mean[1] - std[1] * (Theta[0] + Theta[1] * (X1[0] - mean[0])/std[0]) / Theta[2]
    X2_max = mean[1] - std[1] * (Theta[0] + Theta[1] * (X1[1] - mean[0])/std[0]) / Theta[2]
    return X1, [X2_min, X2_max]

# La fonction qui calcule la combination linéaire entre les caractéristiques
# Vous l'avez déjà vu dans la régression linéaire
def z_1(X, Theta): 
    return np.dot(X, Theta)

# La fonction ségmoid
# Elle doit fonctionner sur des scalaires ou des vecteurs de numpy
def sigmoid(X):
    # Fonction d'activation utilisée pour rendre les valeurs réelles entre 0 et 1 
    return  1 / (1 + np.exp(-X))

def h_1(X, Theta, mean=None, std=None, const=False): 
    norm = (mean is not None) and (std is not None)
    X_pre, mean, std = preparer(X, norm, const, mean=mean, std=std)
    return sigmoid(z_1(X_pre, Theta))

### I-2-  Régression Ridge 

#### I-2-1- La fonction du coût avec régularisation L2

La fonction du coût est donnée par : 

$$ cout(h_\theta(x), y) = - y \log(h_\theta(x)) - (1-y) \log(1 - h_\theta(x))$$

$$J(\theta) = \frac{1}{m} \sum\limits_{i=1}^{m} cout(h_\theta(x^{(i)}), y^{(i)})$$

En appliquant une régularisation **L2** sur cette fonction, on aura : 

$$J(\theta) = \frac{1}{m} \sum\limits_{i=1}^{m} cout(h_\theta(x^{(i)}), y^{(i)}) + \frac{\lambda }{2m} \sum\limits_{j=1}^{n} \theta_j^2$$


**Remarque** Dans les énoncés, la régularisation n'est pas divisée par le nombre des échantillons $m$. Mais, en pratique, on la divise. [Voici des réponses sur stackexchange](https://stats.stackexchange.com/questions/287920/regularisation-why-multiply-by-1-2m) expliquant pourquoi.

In [ ]:
# Voici la fonction du coût utilisée dans la régression logistique 
# H est la prédiction, Y est la valeur réel
def J(H, Y): 
    return - np.mean(Y * np.log(H) + (1-Y) * np.log(1-H))  

# TODO compléter la fonction du coût avec régularisation L2
def J_L2(H, Y, Theta, Lambda=0.):
    return None


# =====================================================
# ====================TEST=============================
# =====================================================
X_t = np.array([[2., -8.], [1., -2.], [1., 2.]])
Y_t = np.array([0., 1., 1.])
Theta_t = np.array([0.5, 0.25])
# J_L2(X_t, Y_t, Theta_t, l=.5) = 0.4398901851987969 + 0.5/(2*3) *((0.5)^2 + (0.25)^2)
#                               = 0.4398901851987969 + 0.026041666666 = 0.4659318518654636

H_t = h_1(X_t, Theta_t)
# Résultat : (0.4398901851987969, 0.4659318518654636)
J_L2(H_t, Y_t, Theta_t), J_L2(H_t, Y_t, Theta_t, Lambda=.5)

#### 1-2-2- Les gradients avec régularisation L2

Les gradients sont calculés par:
$$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum\limits_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_{ij} $$

Voici les gradients régularisées avec **L2** :  
$$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum\limits_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_{ij} + \frac{\lambda }{m} \theta_j$$


In [ ]:
# Voici la fonction de gradient :
def gradient(X, H, Y):
    return  np.dot(X.T, H-Y)/float(len(Y))

# TODO compléter la fonction du gradient avec régularisation L2
# Ici, on peut ne pas utiliser H comme paramètre puisqu'on peut le calculer
# en utilisant X et theta
# Mais, on le fait pour ne pas recalculer H deux fois lors de la descente du gradient 
def gradient_L2(X, H, Y, Theta, Lambda=0.):
    return None


# =====================================================
# ====================TEST=============================
# =====================================================
X_t = np.array([[2., -8.], [1., -2.], [1., 2.]])
Y_t = np.array([0., 1., 1.])
Theta_t = np.array([0.5, 0.25])
H_t = h_1(X_t, Theta_t)
# gradient_L2(X, H, Y, Theta_0, l=0.5) = -0.07701953 + (0.5)/3 * 0.5 = 0.0063138033
# gradient_L2(X, H, Y, Theta_1, l=0.5) = -0.56313807 + (0.25)/3 * 0.5 = -0.521471403

# Résultat : (array([-0.07701953, -0.56313807]), array([ 0.00631381, -0.5214714 ]))
gradient(X_t, H_t, Y_t), gradient_L2(X_t, H_t, Y_t, Theta_t, Lambda=0.5)

#### 1-2-3- Entraînement (algorithme du gradient avec régularisation L2)

Les coéfficients sont mis à jour itérativement en se basant sur le gradient et un pas d'apprentissage $\alpha$. 

$$\theta_j = \theta_j - \alpha \frac{\partial J(\theta)}{\theta_j} $$


In [ ]:
def entrainer_L2(X, Y, norm=True, const=True, nbr_iter=200, alpha=0.1, Lambda=0.1, eps=0.01): 
    
    X_pre, mean, std = preparer(X, norm, const)
    Theta = generer_uns_1(X_pre.shape[1])
    
    couts = []
    thetas = []
    
    for i in range(nbr_iter):
        H = h_1(X_pre, Theta)
        Theta -= alpha * gradient_L2(X_pre, H, Y, Theta, Lambda=Lambda)
        couts.append(J_L2(H, Y, Theta, Lambda=Lambda))
        thetas.append(list(Theta))#pour ne pas ajouter la même référence vers le tableau
        #print(Theta)
        """if i > 1 and abs(couts[-1] - couts[-2]) <= eps:
            return thetas, mean, std, couts"""
        """if i > 1 and couts[-1] > couts[-2] :
            couts.pop()
            thetas.pop()
            return thetas, mean, std, couts"""
    
    return thetas, mean, std, couts

# =====================================================
# ====================TEST=============================
# =====================================================
X_t = np.array([[2., -8.], [1., -2.], [1., 2.]])
Y_t = np.array([0., 1., 1.])
Theta_t = np.array([0.5, 0.25])

thetas1, mean1, std1, couts1 = entrainer_L2(X_t, Y_t)

# Résultat : 
#([0.924218533040175, -1.090846816073148, 1.676832308649455],
# array([ 1.33333333, -2.66666667]),
# array([0.47140452, 4.10960934]),
# 0.15341976582462558,
# 200)
thetas1[-1], mean1, std1, couts1[-1], len(couts1)


### I-3-  Régression Lasso 

#### I-3-1- La fonction du coût avec régularisation L1

La fonction du coût est donnée par : 

$$ cout(h_\theta(x), y) = - y \log(h_\theta(x)) - (1-y) \log(1 - h_\theta(x))$$

$$J(\theta) = \frac{1}{m} \sum\limits_{i=1}^{m} cout(h_\theta(x^{(i)}), y^{(i)})$$

En appliquant une régularisation **L1** sur cette fonction, on aura : 

$$J(\theta) = \frac{1}{m} \sum\limits_{i=1}^{m} cout(h_\theta(x^{(i)}), y^{(i)}) + \frac{\lambda }{m} \sum\limits_{j=1}^{n} |\theta_j|$$

In [ ]:
# TODO compléter la fonction du coût avec régularisation L1
# Souvenez-vous qu'on a déjà définit J(H, Y)
def J_L1(H, Y, Theta, Lambda=0.):
    return None

# =====================================================
# ====================TEST=============================
# =====================================================
X_t = np.array([[2., -8.], [1., -2.], [1., 2.]])
Y_t = np.array([0., 1., 1.])
Theta_t = np.array([0.5, 0.25])
# J_L1(X_t, Y_t, Theta_t, l=.5) = 0.4398901851987969 + 0.5/(3) *( |0.5| + |0.25|)
#                               = 0.4398901851987969 + 0.125 = 0.56489018...

H_t = h_1(X_t, Theta_t)
# Résultat : (0.4398901851987969, 0.5648901851987969)
J_L1(H_t, Y_t, Theta_t), J_L1(H_t, Y_t, Theta_t, Lambda=.5)

#### I-3-2- Entraînement : Iterative soft-thresholding algorithm (ISTA)

Le dérivé de $L1$ est :
$$\frac{\partial L1}{\partial \theta_j} = \frac{\partial \frac{\lambda }{m} \sum\limits_{j=1}^{n} |\theta_j|}{\partial \theta_j} = \frac{\lambda }{m} \frac{\theta_j}{|\theta_j|}$$

Il est clair que le gradient n'est pas définit dans le cas $\theta_j = 0$. 
En appliquant [le sous-différentiel](https://fr.wikipedia.org/wiki/Sous-diff%C3%A9rentiel), on aura : 
$$
\frac{\partial L1}{\partial \theta_j} = 
\begin{cases}
-\frac{\lambda}{m} & \text{si } \theta_j < 0 \\
0 & \text{si } \theta_j = 0 \\
\frac{\lambda}{m} & \text{si } \theta_j > 0 \\
\end{cases}
$$

On ne peut pas utiliser la descente du gradient pour résoudre ça. 
On va utiliser un alorithme de descente du gradient approximative, qui s'appelle $ISTA$. 
$$prox(\theta) = S_{\lambda}(\theta) = 
\begin{cases}
\theta_j + \lambda & \text{si } \theta_j < - \lambda \\
0 & \text{si } -\lambda \le \theta_j \le \lambda \\
\theta_j - \lambda & \text{si } \theta_j > \lambda \\
\end{cases}
$$

**Vous devez implémenter cette fonction**

Les thétas sont mises à jour comme suite : 
$$ \theta = S_{\lambda}(\theta - \frac{\partial J_0}{\partial \theta})$$
Où, $J_0$ est la fonction du coût sans régularisation

Source : [Ryan Tibshirani (2015), Lecture 08: Proximal Gradient Descent](https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/)

In [ ]:
# TODO compléter la fonction soft-thresholding
def S(Theta, Lambda): 
    return None

# =====================================================
# ====================TEST=============================
# =====================================================

# Résultat : array([ 0.4 , -0.15,  0.  ])

Theta_t = np.array([0.5, -0.25, 0.05])

S(Theta_t, 0.1)

In [ ]:
def entrainer_L1(X, Y, norm=True, const=True, nbr_iter=200, alpha=0.1, Lambda=0.1, eps=0.01): 
    
    X_pre, mean, std = preparer(X, norm, const)
    Theta = generer_uns_1(X_pre.shape[1])
    m = len(X_pre)
    
    couts = []
    thetas = []

    for i in range(nbr_iter):
        H = h_1(X_pre, Theta)
        Theta = S(Theta - alpha * gradient(X_pre, H, Y), Lambda/m)
        couts.append(J_L1(H, Y, Theta, Lambda=Lambda))
        thetas.append(list(Theta)) #pour ne pas ajouter le même référence vers le tableau
        #print(couts[-1])
        """if i > 1 and couts[-1] > couts[-2] :
            couts.pop()
            thetas.pop()
            return thetas, mean, std, couts"""
        
        """if i > 1 and abs(couts[-1] - couts[-2]) <= eps:
            return thetas, mean, std, couts"""
    
    return thetas, mean, std, couts

# =====================================================
# ====================TEST=============================
# =====================================================

X_t = np.array([[2., -8.], [1., -2.], [1., 2.]])
Y_t = np.array([0., 1., 1.])
Theta_t = np.array([0.5, 0.25])

thetas1, mean1, std1, couts1 = entrainer_L1(X_t, Y_t)

# Résultat :
#([0.0, -0.260744176157334, 0.24981593638559163],
# array([ 1.33333333, -2.66666667]),
# array([0.47140452, 4.10960934]),
# 0.510021367108139)

thetas1[-1], mean1, std1, couts1[-1]

### I-4- Regrouper les fonctions ensemble 


In [ ]:
class RegLogReg(object): # Régression logistique régularisée
    
    def __init__(self, nbr_iter=100, alpha=.01, Lambda=0.01, norm=True, const=True, reg="l2"): 
        self.nbr_iter = nbr_iter
        self.alpha = alpha
        self.Lambda = Lambda
        self.norm = norm
        self.const = const
        self.reg = reg
    
    def entrainer(self, X, Y): 
        if self.reg == "l1":
            entrainer = entrainer_L1
        else:
            entrainer = entrainer_L2
            
        self.thetas, self.mean, self.std, self.couts = entrainer(X, Y, 
                                                                  nbr_iter=self.nbr_iter, 
                                                                  alpha=self.alpha, 
                                                                  Lambda=self.Lambda, 
                                                                  norm=self.norm, 
                                                                  const=self.const)
        
    # La prédiction
    # si prob=True elle rend un vecteur de probabilités
    # sinon elle rend une vecteur de 1 et 0
    def predire(self, X, prob=False, seuil=0.5, it=-1):
        H = h_1(X, self.thetas[it], self.mean, self.std, self.const)
        if prob:
            return H
        return (H >= seuil).astype(int)

# =====================================================
# ====================TEST=============================
# =====================================================
X_t = np.array([[2., -8., 1.], [1., -2., 3.], [1., 2., 4.], [5., -2., -1.]]) # 4X3
Y_t = np.array([2., -1., 1., 3.]) # 4X1
Theta_t = np.array([0.5, 0.3, 0.2]) # 3X1

reg2log = RegLogReg()
reg2log.entrainer(X_t, Y_t)
p_l2 = reg2log.predire(np.array([[2., -8., 1.], [1., -2., 3.]]))
nbr_it_l2 = len(reg2log.couts)

reg2log = RegLogReg(reg="l1")
reg2log.entrainer(X_t, Y_t)
p_l1 = reg2log.predire(np.array([[2., -8., 1.], [1., -2., 3.]]))
nbr_it_l1 = len(reg2log.couts)

reg2log = RegLogReg(Lambda=0)
reg2log.entrainer(X_t, Y_t)
p_noreg = reg2log.predire(np.array([[2., -8., 1.], [1., -2., 3.]]))
nbr_it_noreg = len(reg2log.couts)

#Résultat : (array([1, 0]), 100, array([1, 0]), 100, array([1, 0]), 100)
p_l2, nbr_it_l2, p_l1, nbr_it_l1, p_noreg, nbr_it_noreg

### I-5- Comparison 

On va comparer entre la régression logistique (gradient descent), la régression logistique avec régularisation **L2** (gradient descent) et la régression logistique avec régularisation **L1** (ISTA). 
Pour ce faire, on va utiliser le dataset [Diabetics prediction using logistic regression](https://www.kaggle.com/kandij/diabetes-dataset). 

In [ ]:
# diabetes2
diabetes = pd.read_csv("datasets/diabetes2.csv") 
X_diabetes = diabetes.iloc[:, :-1].values  
Y_diabetes = diabetes.iloc[:, -1].values

# Cette configuration est mise en place comme ceci exprès
# C'est pour tester le cas où la régression est difavorisée
NBR_TEST = 240
# Supposant que les 30% premières lignes sont pour le test et le reste pour l'entraînement
X_test = X_diabetes[-NBR_TEST:, :] # 30% ou plus
Y_test = Y_diabetes[-NBR_TEST:]

X_train = X_diabetes[:-NBR_TEST, :] 
Y_train = Y_diabetes[:-NBR_TEST]

diabetes.head()

In [ ]:
ALPHA = .2
LAMBDA = 20.
NBR_ITER=100

reg2log_L2 = RegLogReg(alpha=ALPHA, Lambda=LAMBDA, nbr_iter=NBR_ITER, const=True) # par défaut reg="L2"
reg2log_L2.entrainer(X_train, Y_train)

reg2log_L1 = RegLogReg(reg="l1", alpha=ALPHA, Lambda=LAMBDA, nbr_iter=NBR_ITER, const=True) # L1
reg2log_L1.entrainer(X_train, Y_train)

reg2log = RegLogReg(alpha=ALPHA, Lambda=0, nbr_iter=NBR_ITER, const=True) # L2 avec lambda = 0 devient une regression normale
reg2log.entrainer(X_train, Y_train)

from sklearn.metrics import recall_score, precision_score

recalls_train = {
    "reg" : [],
    "reg-L2": [],
    "reg-L1": []
}

precisions_train = {
    "reg" : [],
    "reg-L2": [],
    "reg-L1": []
}

recalls_test = {
    "reg" : [],
    "reg-L2": [],
    "reg-L1": []
}

precisions_test = {
    "reg" : [],
    "reg-L2": [],
    "reg-L1": []
}

for it in range(NBR_ITER): 
    
    Y_train_reg = reg2log.predire(X_train, it=it)
    Y_train_reg_L2 = reg2log_L2.predire(X_train, it=it)
    Y_train_reg_L1 = reg2log_L1.predire(X_train, it=it)
    
    recalls_train["reg"].append(recall_score(Y_train, Y_train_reg))
    recalls_train["reg-L2"].append(recall_score(Y_train, Y_train_reg_L2))
    recalls_train["reg-L1"].append(recall_score(Y_train, Y_train_reg_L1))
    
    precisions_train["reg"].append(precision_score(Y_train, Y_train_reg))
    precisions_train["reg-L2"].append(precision_score(Y_train, Y_train_reg_L2))
    precisions_train["reg-L1"].append(precision_score(Y_train, Y_train_reg_L1))
    
    Y_test_reg = reg2log.predire(X_test, it=it)
    Y_test_reg_L2 = reg2log_L2.predire(X_test, it=it)
    Y_test_reg_L1 = reg2log_L1.predire(X_test, it=it)
    
    recalls_test["reg"].append(recall_score(Y_test, Y_test_reg))
    recalls_test["reg-L2"].append(recall_score(Y_test, Y_test_reg_L2))
    recalls_test["reg-L1"].append(recall_score(Y_test, Y_test_reg_L1))
    
    precisions_test["reg"].append(precision_score(Y_test, Y_test_reg))
    precisions_test["reg-L2"].append(precision_score(Y_test, Y_test_reg_L2))
    precisions_test["reg-L1"].append(precision_score(Y_test, Y_test_reg_L1))

print("FIN")

#### I-5-1- Convergence du coût d'entraînement 

**Analyser les résultats** 

Ici, on ne peut pas comparer entre les coûts en terme de valeur, puisque les trois algorithmes ont une fonction du coût différente. 
Mais, on veut seulement comparer entre la rapidité de convergence de chaque algorithme vers sa solution optimale. 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.suptitle("Convergence des couts")
fig.set_figwidth(15)
ax1.plot(range(NBR_ITER), reg2log.couts, color="blue", label="Reg")
ax2.plot(range(NBR_ITER), reg2log_L1.couts, color="red", label="Reg-L1")
ax3.plot(range(NBR_ITER), reg2log_L2.couts, color="green", label="Reg-L2")

ax1.legend()
ax1.set_title("Regression")
ax2.legend()
ax2.set_title("Regression+L1")
ax3.legend()
ax3.set_title("Regression+L2")

#plt.plot(range(nbr_it[0]), reg2log.couts, color="blue", label="Reg")
#plt.plot(range(nbr_it[1]), reg2log_L1.couts, color="red", label="Reg-L1")
#plt.plot(range(nbr_it[2]), reg2log_L2.couts, color="green", label="Reg-L2")
#plt.ylabel("Couts")
#plt.xlabel("Iterations")
#plt.legend()
plt.show()

#### I-5-2- Performance de l'entraînement et de test selon le nombre des itérations

**Analyser les résultats** 

Ici, on veut savoir dans chaque itération, quelle est le rappel et la précision de l'entraînement. 
Aussi, combien sera le rappel et la précision du test si on arrête l'entraînement après *k* itérations.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
fig.suptitle("Les rappels et les precisions de l'entrainement et de test pour chaque iteration")
fig.set_figwidth(15)
    
ax1.plot(range(NBR_ITER), recalls_train["reg"], color="blue", label="Rappel entrainement")
ax1.plot(range(NBR_ITER), recalls_test["reg"], color="red", label="Rappel test")
ax1.plot(range(NBR_ITER), precisions_train["reg"], color="cyan", label="Precision entrainement")
ax1.plot(range(NBR_ITER), precisions_test["reg"], color="orange", label="Precision test")
ax1.legend()
ax1.set_title("Regression")
ax1.grid()

ax3.plot(range(NBR_ITER), recalls_train["reg-L1"], color="blue", label="Rappel entrainement")
ax3.plot(range(NBR_ITER), recalls_test["reg-L1"], color="red", label="Rappel test")
ax3.plot(range(NBR_ITER), precisions_train["reg-L1"], color="cyan", label="Precision entrainement")
ax3.plot(range(NBR_ITER), precisions_test["reg-L1"], color="orange", label="Precision test")
ax3.legend()
ax3.set_title("Regression+L1")
ax3.grid()

ax2.plot(range(NBR_ITER), recalls_train["reg-L2"], color="blue", label="Rappel entrainement")
ax2.plot(range(NBR_ITER), recalls_test["reg-L2"], color="red", label="test")
ax2.plot(range(NBR_ITER), precisions_train["reg-L2"], color="cyan", label="Precision entrainement")
ax2.plot(range(NBR_ITER), precisions_test["reg-L2"], color="orange", label="Precision test")
ax2.legend()
ax2.set_title("Regression+L2")
ax2.grid()

plt.show()

#### I-5-3- Les paramètres (thétas)

**Analyser les résultats** 

Ici, on veut analyser la convergence des paramètres (thétas). 
On veut, aussi, savoir si les fonctions de régularisation peuvent servir comme fonction de filtrage des attributs. 
En plus, on veut savoir si les algorithmes favorisent des attributs sur d'autres.

**Remarque** Tous les paramètres ont été fixés à 1. Si, on les fixe à 0, LASSO ne les met pas à jour à cause de la condition $-\lambda \le \theta_{new} \le \lambda$ dans **ISTA**.

In [ ]:
labels = list(diabetes.columns)
labels.pop()
labels = ["Theta0"] + labels
fig, (ax1, ax2, ax3) = plt.subplots(3,1)
fig.suptitle("Convergence des thetas")
fig.set_figwidth(7)
fig.set_figheight(12)
for i in range(len(labels)): 
    ax1.plot(range(NBR_ITER), np.array(reg2log.thetas)[:, i], label=labels[i])
    ax3.plot(range(NBR_ITER), np.array(reg2log_L1.thetas)[:, i], label=labels[i])
    ax2.plot(range(NBR_ITER), np.array(reg2log_L2.thetas)[:, i], label=labels[i])
ax1.legend()
ax1.set_title("Regression")
ax1.grid()
ax3.legend()
ax3.set_title("Regression+L1")
ax3.grid()
ax2.legend()
ax2.set_title("Regression+L2")
ax2.grid()

plt.show()

## II- Sélection d'attributs

La sélection des attributs par filtrage : 
- **Régression** sortie numérique
   - **attributs numériques** 
      - Pearson's correlation coefficient (linear)
      - Spearman's rank coefficient (nonlinear)
   - **attributs catégoriques**
      - ANOVA correlation coefficient (linear)
      - Kendall's rank coefficient (nonlinear)

- **Classement** sortie catégorique
   - **attributs numériques** 
      - ANOVA correlation coefficient (linear)
      - Kendall's rank coefficient (nonlinear)
   - **attributs catégoriques**
      - Chi-Squared test (contingency tables)
      - Mutual Information

Source : [Jason Brownlee (2019), How to Choose a Feature Selection Method For Machine Learning](https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/)

### II-1- Implémentation de ANOVA (one-way)


#### II-1-1- One way F value pour un seul attribut

Le nombre des échantillons pour chaque classes est différent. 
Pour cette raison, on va implémenter ANOVA for a CRD (COMPLETELY RANDOM DESIGN). 
On va implémenter l'algorithme expliqué dans [Richard Horsly (2012) COMPLETELY RANDOM DESIGN (CRD)](https://www.ndsu.edu/faculty/horsley/CRD.pdf). 
Pour valider l'implémentation, on va utiliser les mêmes données de la page 10.

pour un attribut donné ($A$), on divise l'ensemble de ses échantillons sur $N$ ensembles, où $N$ est le nombre des classes. 

$$CF = \frac{(\sum_{ij} A_{ij})^2}{\sum_j |A_j|} $$
Où $\sum_{ij} A_{ij}$ est la somme de toutes les valeurs de toutes les classes, 
$\sum_j |A_j|$ est le nombre total des échantillons (de toutes les classes)

$$ TotalSS = \sum_{ij} A_{ij}^2 - CF$$
Où $\sum_{ij} A_{ij}^2$ est les somme des carrées de toutes les valeurs

$$ TreatmentSS = \sum_j \frac{(\sum_{i} A_{ij})^2}{|A_j|} - CF$$
Où $\sum_{i} A_{ij}$ est la somme des valeurs de la classe $j$ et $|A_j|$ est le nombre des éléments dans la classe $j$

$$ ErrorSS = TotalSS - TreatmentSS $$


In [ ]:
# TODO compléter cette fonction
def f_oneway(A): 
    # A est une liste de N listes où N est le nombre des classes
    # Ces listes n'ont pas la même taille
    nbrs = np.array([len(a) for a in A]) # nombre des échantillons de chaque classe
    sommes = np.array([np.sum(a) for a in A]) # somme des valeurs de chaque classe
    sommes_caree = np.sum([np.sum(np.array(a)**2) for a in A]) # somme des valeurs carrées de toutes les valeurs
    
    # Compléter ici
    #====================================
    CF = None
    TotalSS = None
    TreatmentSS = None
    ErrorSS = None
    #====================================
    
    MSB = TreatmentSS / (len(A) - 1) 
    MSW =  ErrorSS/(nbrs.sum() - len(A))
    return MSB/MSW


# =====================================================
# ====================TEST=============================
# =====================================================

AA = [[2., 2.2, 1.8, 2.3, 1.7], [1.7, 1.9, 1.5], [2., 2.4, 2.7, 2.5, 2.4], [2.1, 2.2, 2.2, 1.9]]
# Résultat : 6.418894830659841 (voir page 11 step 6)
f_oneway(AA)

#### II-1-2- One way F value pour tous les attribues

**Rien à programmer ou analyser ici**

Ici, on va calculer ANOVA pour chaque attribut. 
Les attributs avec des grandes valeurs ANOVA sont meilleurs. 



In [ ]:
def anova_f(X, Y): 
    
    C = np.unique(Y)
    F = []
    for j in range(X.shape[1]): 
        A = X[:, j]
        F.append(f_oneway([A[Y==c] for c in np.unique(Y)]))
    return F


# pour valider, on va utiliser f_classif de scikit-learn 
# https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/feature_selection/_univariate_selection.py#L42
from sklearn.feature_selection import f_classif
f, a = f_classif(X_train, Y_train)
# la première liste est celle de scikit-learn et la deuxième est la notre
f, np.array(anova_f(X_train, Y_train))

### II-2- Régression logistique et  sélection d'attributs

Comparison entre régression logistique avant et après filtrage (en terme de rapidité de convergence et en terme de performance)

#### II-2-1- Analyse des attributs 

**Analyser les résultats** 

Quels sont les attributs les plus importants d'après vous ? pourquoi?

In [ ]:
F_diabetes = anova_f(X_train, Y_train)
labels_diabetes = list(diabetes.columns)
labels_diabetes.pop()
x_pos = range(len(F_diabetes))
plt.figure(figsize=(15,5))
plt.bar(labels_diabetes, F_diabetes, color="green")
plt.xlabel("Les attributs")
plt.ylabel("ANOVA F value")
plt.title("Comparison entre les attributs de diabetes2 en terme de ANOVA")

plt.xticks(x_pos, labels_diabetes)

plt.show()


#### II-2-2- Filtrage des attribits et convergence

**Analyser les résultats** 

Ici, on va utiliser les 5 attributs ayant les plus grandes ANOVA F valeurs

In [ ]:
select = [True, True, False, False, False, True, True, True]
reg2log_filt = RegLogReg(alpha=ALPHA, Lambda=0, nbr_iter=NBR_ITER, const=True) # L2 avec lambda = 0 devient une regression normale
reg2log_filt.entrainer(X_train[:, select], Y_train)

plt.plot(range(NBR_ITER), reg2log.couts, color="blue", label="Reg")
plt.plot(range(NBR_ITER), reg2log_filt.couts, color="red", label="Reg Filter")
plt.xlabel("nombre des iterations")
plt.ylabel("cout")
plt.legend()
plt.title("les convergences de la regression sans et avec filtrage")

plt.show()

#### II-2-3- Filtrage des attribits et performance

**Analyser les résultats** 

Ici, on va comparer entre la performance de la régression sans et avec filtrage. 
Pour rendre la comparison plus intéressante, on va aussi comarer avec LASSO

In [ ]:
recalls_train["reg-Filt"] = []
precisions_train["reg-Filt"] = []
recalls_test["reg-Filt"] = []
precisions_test["reg-Filt"] = []
for it in range(NBR_ITER): 
    Y_train_regfilt = reg2log_filt.predire(X_train[:, select], it=it)
    Y_test_regfilt = reg2log_filt.predire(X_test[:, select], it=it)
    recalls_train["reg-Filt"].append(recall_score(Y_train, Y_train_regfilt))
    precisions_train["reg-Filt"].append(precision_score(Y_train, Y_train_regfilt))
    recalls_test["reg-Filt"].append(recall_score(Y_test, Y_test_regfilt))
    precisions_test["reg-Filt"].append(precision_score(Y_test, Y_test_regfilt))


fig, (ax1, ax2, ax3) = plt.subplots(1,3)
fig.suptitle("Les rappels et les precisions de l'entrainement et de test pour chaque iteration")
fig.set_figwidth(15)
    
ax1.plot(range(NBR_ITER), recalls_train["reg"], color="blue", label="Rappel entrainement")
ax1.plot(range(NBR_ITER), recalls_test["reg"], color="red", label="Rappel test")
ax1.plot(range(NBR_ITER), precisions_train["reg"], color="cyan", label="Precision entrainement")
ax1.plot(range(NBR_ITER), precisions_test["reg"], color="orange", label="Precision test")
ax1.legend()
ax1.set_title("Regression")
ax1.grid()

ax3.plot(range(NBR_ITER), recalls_train["reg-L1"], color="blue", label="Rappel entrainement")
ax3.plot(range(NBR_ITER), recalls_test["reg-L1"], color="red", label="Rappel test")
ax3.plot(range(NBR_ITER), precisions_train["reg-L1"], color="cyan", label="Precision entrainement")
ax3.plot(range(NBR_ITER), precisions_test["reg-L1"], color="orange", label="Precision test")
ax3.legend()
ax3.set_title("Regression+L1")
ax3.grid()

ax2.plot(range(NBR_ITER), recalls_train["reg-Filt"], color="blue", label="Rappel entrainement")
ax2.plot(range(NBR_ITER), recalls_test["reg-Filt"], color="red", label="test")
ax2.plot(range(NBR_ITER), precisions_train["reg-Filt"], color="cyan", label="Precision entrainement")
ax2.plot(range(NBR_ITER), precisions_test["reg-Filt"], color="orange", label="Precision test")
ax2.legend()
ax2.set_title("Regression+Filter")
ax2.grid()

plt.show()